# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115329e+02     1.719396e+00
 * time: 0.11944198608398438
     1     1.028096e+01     1.015483e+00
 * time: 0.9044029712677002
     2    -1.231599e+01     1.219490e+00
 * time: 1.0385658740997314
     3    -3.476916e+01     8.779338e-01
 * time: 1.1894049644470215
     4    -4.859381e+01     6.385843e-01
 * time: 1.3342149257659912
     5    -5.739280e+01     1.863560e-01
 * time: 1.4757139682769775
     6    -5.996100e+01     7.427854e-02
 * time: 1.5808610916137695
     7    -6.083658e+01     5.840308e-02
 * time: 1.69193696975708
     8    -6.125606e+01     6.195430e-02
 * time: 1.796396017074585
     9    -6.159090e+01     3.554498e-02
 * time: 1.9089910984039307
    10    -6.183699e+01     2.766577e-02
 * time: 2.0219690799713135
    11    -6.196633e+01     2.174162e-02
 * time: 2.126115083694458
    12    -6.204627e+01     1.968356e-02
 * time: 2.2364749908447266
    13    -6.209726e+01     1.578394e-02
 * time: 2.3403899669647

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557703
    AtomicNonlocal      14.8522673
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485375 
    Xc                  -19.3336821

    total               -62.261666456939
